$$
\text{PET} = 0.0023 \times R_A \times \sqrt{T_{max} - T_{min}} \times (T + 17.8)
$$

- $K_R = 0.16$ for "interior" locations, where air masses are not strongly influenced by a large water body
- $K_R = 0.19$ otherwise.

$R_A$ should be multiplied by 0.408 to convert it from [MJ m-2 day-1] to [mm day-1].

In [ ]:
# TODO Describe the importance of docstrings

import numpy as np

def toa_radiation(latitude, doy):
    '''
    Top-of-atmosphere (TOA) radiation for a given latitude (L) and day of year
    (DOY) can be calculated as:

    R = ((24 * 60) / pi) * G * d * (w * sin(L) * sin(D) + cos(L) * cos(D) * sin(w))

    Where G is the solar constant, 0.0820 [MJ m-2 day-1]; d is the earth-sun
    distance; w is the sunset hour angle; and D is the solar declination angle.
    
    For more information, consult the FAO documentation:

        https://www.fao.org/4/X0490E/x0490e07.htm#radiation
    
    Parameters
    ----------
    latitude : float
        The latitude on earth, in degrees
    doy : int
        The day of the year (DOY), an integer on [1,366]
    
    Returns
    -------
    Number
        Top-of-atmosphere (TOA) radiation, in [MJ m-2 day-1]
    '''
    solar_constant = 0.0820 # [MJ m-2 day-1]
    pi = 3.14159
    # Convert latitude from degrees to radians
    lat_radians = np.deg2rad(latitude)
    # Earth-Sun distance, as a function of day-of-year (DOY)
    earth_sun_dist = 1 + 0.0033 * np.cos(doy * ((2 * pi) / 365))
    # Solar declination, as a function of DOY
    declination = 0.409 * np.sin(doy * ((2 * pi) / 365) - 1.39)
    # Sunset hour angle
    sunset_hour_angle = np.arccos(-np.tan(lat_radians) * np.tan(declination))
    return ((24 * 60) / pi) * solar_constant * earth_sun_dist *\
        (sunset_hour_angle * np.sin(lat_radians) * np.sin(declination) +
            np.cos(lat_radians) * np.cos(declination) * np.sin(sunset_hour_angle))

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2024-01-01", "2024-05-31"))

In [ ]:
# Could take about 1 minute on a broadband connection
earthaccess.download(results, 'data_raw/MERRA2')

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*.nc4')
ds

In [ ]:
ds['T2MMEAN']

In [ ]:
# TODO Figuring out what the coordinates are

ds.coords

In [ ]:
# TODO Re-chunking the data; give example of "what if" we were interested in calculating trends

ds = xr.open_mfdataset('./data_raw/MERRA2/*.nc4')
ds['T2MMEAN']

In [ ]:
# TODO Even though we asked for 122 elements along the "time" dimension, because the `chunks` argument is
#    applied on a *per-file basis,* it can't build chunks of that size when reading in the data

ds = xr.open_mfdataset('./data_raw/MERRA2/*.nc4', chunks = {'time': 122})
ds['T2MMEAN']

In [ ]:
# TODO We can, however, create chunks within each file

ds = xr.open_mfdataset('./data_raw/MERRA2/*.nc4', chunks = {'lat': 182, 'lon': 288})
ds['T2MMEAN']

In [ ]:
# TODO Re-chunking the data *after* loading is generally inefficient, but might be necessary; 
#    give example of "what if" we were interested in calculating trends

ds = xr.open_mfdataset('./data_raw/MERRA2/*.nc4')
ds = ds.chunk({'time': 122})
ds['T2MMEAN']

In [ ]:
# TODO But today, the function we're going to apply to the data doesn't depend on neighboring pixels or groups of pixels over time, so we don't care so much about how the chunks are formed

ds = xr.open_mfdataset('./data_raw/MERRA2/*.nc4', chunks = 'auto')
ds['T2MMEAN']